### 평균 해시 매칭
* 참고: https://dacon.io/codeshare/5106

In [2]:
# 라이브러리 불러오기
import cv2

In [9]:
#영상 읽어서 출력해보기
img = cv2.imread('./images/pistol.jpg')
cv2.namedWindow('pistol', cv2.WINDOW_GUI_NORMAL)
cv2.imshow('pistol', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

1. 평균 해시값 구하기

In [ ]:
# 그레이 스케일로 변환
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [4]:
# 8x8 크기로 축소 ---①
gray = cv2.resize(gray, (16,16))

In [5]:
# 영상의 평균값 구하기 ---②
avg = gray.mean()

In [6]:
# 평균값을 기준으로 0과 1로 변환 ---③
bin = 1 * (gray > avg)
print(bin)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 0 0 0 1 1 1 1 1 1 1]
 [1 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1]]


In [7]:
# 2진수 문자열을 16진수 문자열로 변환 ---④
dhash = []
for row in bin.tolist():
    s = ''.join([str(i) for i in row])
    dhash.append('%02x'%(int(s,2)))
dhash = ''.join(dhash)
print(dhash)

ffff8000800080008000813fc1ffc1ffc07fc3ffc7ffc7ff87ff87ff87ffc7ff


2. 템플릿 매칭

* 템플릿 매칭은 어떤 물체가 있는 영상을 준비해두고 그 물체가 포함되어 있을 것이라고 예상할 수 있는 입력 영상과 비교해서 물체가 매칭되는 위치를 찾는 것
* OpenCV는 템플릿 매칭 cv2.matchTemplate()함수를 제공  
    + result = cv2.matchTemplate(img, templ, method[, result, mask])

        - img : 입력 영상
        - templ : 템플릿 영상
        - method : 매칭 메서드
            cv2.TM_SQDIFF : 제곱 차이 매칭, 완벽 매칭: 0, 나쁜 매칭: 큰 값  
            cv2.TM_SQDIFF_NORMED : 제곱 차이 매칭의 정규화  
            cv2.TM_CCORR : 상관관계 매칭, 완벽 매칭: 큰 값, 나쁜 매칭: 0  
            cv2.TM_CCORR_NORMED : 상관관계 매칭의 정규화  
            cv2.TM_CCOEFF : 상관계수 매칭, 완벽 매칭: 1, 나쁜 매칭: -1  
        result : 매칭 결과 2차원 배열  
        mask : TM_SQDIFF, TM_CCORR_NORMED인 경우 사용할 마스크  

    + minVal, maxVal, minLoc, maxLoc = **cv2.minMaxLoc**( src[, mask])  

        - src : 입력 1채널 배열
        - minVal, maxVal : 배열 전체에서 최소값, 최대값
        - minLoc, maxLoc : 최소값과 최대값의 좌표(x, y)

In [11]:
import cv2
import numpy as np

# 입력이미지와 템플릿 이미지 읽기
# 이미지 다운로드: https://data.caltech.edu/records/mzrjq-6wc02
img = cv2.imread('./images/figures.jpg')
template = cv2.imread('./images/pica.jpg')
th, tw = template.shape[:2]
cv2.imshow('template', template)

# 3가지 매칭 메서드 순회
methods = ['cv2.TM_CCOEFF_NORMED', 'cv2.TM_CCORR_NORMED', 'cv2.TM_SQDIFF_NORMED']
for i, method_name in enumerate(methods):
    img_draw = img.copy()
    method = eval(method_name)
    # 템플릿 매칭   ---①
    res = cv2.matchTemplate(img, template, method)
    # 최대, 최소값과 그 좌표 구하기 ---②
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    print(method_name, min_val, max_val, min_loc, max_loc)

    # TM_SQDIFF의 경우 최소값이 좋은 매칭, 나머지는 그 반대 ---③
    if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
        top_left = min_loc
        match_val = min_val
    else:
        top_left = max_loc
        match_val = max_val
    # 매칭 좌표 구해서 사각형 표시   ---④      
    bottom_right = (top_left[0] + tw, top_left[1] + th)
    cv2.rectangle(img_draw, top_left, bottom_right, (0,0,255),2)
    # 매칭 포인트 표시 ---⑤
    cv2.putText(img_draw, str(match_val), top_left, cv2.FONT_HERSHEY_PLAIN, 2,(0,255,0), 1, cv2.LINE_AA)
    cv2.imshow(method_name, img_draw)
cv2.waitKey(0)
cv2.destroyAllWindows()    

cv2.TM_CCOEFF_NORMED -0.34496572613716125 0.9997499585151672 (323, 246) (454, 278)
cv2.TM_CCORR_NORMED 0.7524874806404114 0.9999744296073914 (249, 53) (454, 278)
cv2.TM_SQDIFF_NORMED 5.1157374400645494e-05 0.5842239856719971 (454, 278) (249, 52)
